In [ ]:
import gym
from gym import Env, spaces

import numpy as np
import seaborn as sns
from plotly.offline import iplot
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import random
import networkx as nx
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.legend_handler import HandlerPatch,HandlerTuple
import matplotlib.patches as patches
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy

import gymnasium as gym
from gymnasium import Env, spaces

from sklearn import preprocessing
from matplotlib.patches import FancyArrowPatch

Configurar o Enxame


In [2]:

num_drones = 4
limite_y = 80
limite_x = 100
alcance_comunicacao_jammer = 25
alcance_comunicacao_nos = 5000
B_Hz = 2.4e9
posicoes = np.array([[18, 52], [33, 20], [44, 15], [48, 25]])
posicao_jammer = np.array([40, 70])

In [3]:
# Definir a subgrade dentro da área maior
area_size = (100, 100)
subgrade_start = (25, 20)  # Ponto inicial da subgrade (x, y)
subgrade_size = (50, 40)   # Tamanho da subgrade (largura, altura)
resolution = 5             # Resolução da subgrade (distância entre pontos)

# Gerar os pontos discretizados na subgrade
x_points = np.arange(subgrade_start[0], subgrade_start[0] + subgrade_size[0] + 1, resolution)
y_points = np.arange(subgrade_start[1], subgrade_start[1] + subgrade_size[1] + 1, resolution)
grid_points = np.array(np.meshgrid(x_points, y_points)).T.reshape(-1, 2)


VIZUALIZAÇÃO DAS DIREÇÕES DAS ANTENAS ESCOLHIDAS ALEATORIAMENTE

In [ ]:
# Função para calcular a distância entre duas posições
def distancia(pos1, pos2):
    return np.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

# Função para encontrar vizinhos dentro do alcance de comunicação
def encontra_vizinhos(posicoes, alcance_comunicacao_nos):
    num_drones = posicoes.shape[0]
    vizinhos = {}
    for i in range(num_drones):
        vizinhos[i] = []
        for j in range(num_drones):
            if i != j and distancia(posicoes[i], posicoes[j]) <= alcance_comunicacao_nos:
                vizinhos[i].append(j)
    return vizinhos

# Função para verificar quais drones são afetados pelo jammer
def verifica_jammer(posicoes, posicao_jammer, alcance_comunicacao_jammer):
    afetados_pelo_jammer = []
    for i in range(posicoes.shape[0]):
        if distancia(posicoes[i], posicao_jammer) <= alcance_comunicacao_jammer:
            afetados_pelo_jammer.append(i)
    return afetados_pelo_jammer

# Gera direções aleatórias para as antenas
direcoes_antena = np.random.uniform(0, 360, size=num_drones)
# direcoes_antena = [278, 102, 235, 312, 182, 242]
afetados_pelo_jammer=verifica_jammer(posicoes, posicao_jammer, alcance_comunicacao_jammer)
# Comprimento das setas para indicar a direção das antenas
comprimento_seta = 5



# Visualização usando Matplotlib
def render_v2(posicoes, direcoes_antena, posicao_jammer, comprimento_seta=5, area_size=(100, 100)):
    afetados_pelo_jammer = verifica_jammer(posicoes, posicao_jammer, alcance_comunicacao_jammer)

    fig, ax = plt.subplots(figsize=(9, 4))

    # Desenha os drones no gráfico
    for i, pos in enumerate(posicoes):
        ax.scatter(pos[0], pos[1], color='blue', s=100, label='Drone' if i == 0 else "")
        ax.text(pos[0], pos[1] - 2, f'{i}', horizontalalignment='center', color='white', fontweight='bold')

    # Desenha as setas para a direção das antenas
    for i, direcao in enumerate(direcoes_antena):
        direcao_rad = np.radians(direcao)  # Converte para radianos
        dx = comprimento_seta * np.cos(direcao_rad)  # Deslocamento em x
        dy = comprimento_seta * np.sin(direcao_rad)  # Deslocamento em y
        
        ax.arrow(
            posicoes[i][0], posicoes[i][1], 
            dx, dy, 
            head_width=2, head_length=2, fc='b', ec='b'
        )  # Seta azul para indicar a direção da antena

    # Desenha o jammer
    ax.scatter(posicao_jammer[0], posicao_jammer[1], color='red', s=100, marker='o', label='Jammer')

    # Configurações finais do gráfico
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_xlim(0, area_size[0])
    ax.set_ylim(0, area_size[1])
    ax.set_yticks(np.arange(0, area_size[1]+1, 20))
    plt.title('Direções das Antenas UAV')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize='small')

    plt.show()

render_v2(posicoes, direcoes_antena, posicao_jammer, comprimento_seta)


CALCULO DE TODOS OS ANGULOS A USAR

In [ ]:
import numpy as np

# Função para calcular o ângulo entre dois nós em relação à direção da antena
def angulo_entre_nos(pos1, direcao_antena, pos2):
    # Vetor entre os dois nós
    delta_x = pos2[0] - pos1[0]
    delta_y = pos2[1] - pos1[1]
    
    # Direção do vetor em graus
    direcao_vetor_rad = np.arctan2(delta_y, delta_x)
    direcao_vetor_deg = np.degrees(direcao_vetor_rad)
    
    # Diferença entre a direção do vetor e a direção da antena
    angulo = direcao_vetor_deg - direcao_antena
    
    # Ajustar para o intervalo de 0 a 360 graus
    angulo = angulo % 360  # Usa módulo para garantir que o valor esteja entre 0 e 360
    angulo_arredondado = round(angulo)
    
    return angulo_arredondado

# Posições dos drones
# posicoes = np.array([[16, 30], [18, 48], [44, 15], [23, 73], [45, 73], [50, 50]])

# Direções aleatórias das antenas
# direcoes_antena = np.random.uniform(0, 360, size=posicoes.shape[0])

# Matriz para armazenar os ângulos
angulos_matriz = np.zeros((posicoes.shape[0], posicoes.shape[0]))

# Calcular os ângulos para todas as combinações de drones
for i in range(posicoes.shape[0]):
    for j in range(posicoes.shape[0]):
        if i != j:
            angulos_matriz[i, j] = angulo_entre_nos(posicoes[i], direcoes_antena[i], posicoes[j])

# Apresentar os ângulos entre drones sem valores negativos
for i in range(posicoes.shape[0]):
    print(f"Drones que o drone {i} 'vê' com um ângulo:")
    for j in range(posicoes.shape[0]):
        if i != j:
            angulo_arredondado = int(round(angulos_matriz[i, j]))  # Arredondar para unidade
            print(f"  Com o drone {j}: {angulo_arredondado} graus")

PARAMETROS 


In [6]:
Ptx_dBm = 20 
f = 2.4e9 
B_Hz= 2.4e9
d0=1
gamma=2
sigma = 0
c=3e8
lambda_m=c/f
L0=30
potencia_jammer_dBm =100

# Ler o arquivo de ganhos para criar uma tabela de busca
ganhos_df = pd.read_csv('ganhos.csv')

In [ ]:
angulos = np.deg2rad(ganhos_df['angulo'])  # Converter graus para radianos
ganhos = ganhos_df['ganho']

# Criar um gráfico polar
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(angulos, ganhos, linestyle='-', linewidth=1, label='Ganho da Antena')

# Ajustar o layout
ax.set_theta_zero_location('N')  # Configurar a direção Norte como o topo do gráfico
ax.set_theta_direction(-1)  # Configurar a direção dos ângulos para sentido horário
ax.set_rmax(max(ganhos) + 1)  # Ajustar o raio máximo para melhor visualização
ax.set_rticks(np.linspace(min(ganhos), max(ganhos), num=5))  # Definir os ticks para o raio
ax.set_rlabel_position(45)  # Posicionar os labels do raio

# Configurar os ticks para o ângulo
ax.set_xticks(np.deg2rad(np.arange(0, 360, 30)))  # Definir marcas de ângulo a cada 30 graus

# Adicionar legenda e título
ax.legend()
ax.set_title('Ganho da Antena por Ângulo', va='bottom')

# Mostrar o gráfico
plt.show()

In [ ]:
def busca_ganhos( angulo):
    # Arredondar para o ângulo inteiro mais próximo
    angulo_ajustado = int(round(angulo))
    
    if angulo_ajustado==360:
        angulo_ajustado=0
    
    # Encontrar o ganho correspondente ao ângulo ajustado
    ganho = ganhos_df.loc[ganhos_df['angulo'] == angulo_ajustado, 'ganho'].iloc[0]
    
    return ganho



# Função para calcular os ganhos de transmissão e recepção
def calcula_ganhos(posicoes, direcoes_antena, indice_transmissor, indice_receptor, ganhos_df):
    # Verificar se o índice está dentro do limite de direcoes_antena
    if indice_transmissor >= len(direcoes_antena) or indice_receptor >= len(direcoes_antena):
        print(f"Erro: índice fora dos limites de direcoes_antena. Transmissor: {indice_transmissor}, Receptor: {indice_receptor}, Tamanho de direcoes_antena: {len(direcoes_antena)}")
        return 0, 0  # Retorne ganhos padrão ou levante um erro

    # Obter posições e direções
    pos1 = posicoes[indice_transmissor]
    pos2 = posicoes[indice_receptor]
    direcao_antena1 = direcoes_antena[indice_transmissor]
    direcao_antena2 = direcoes_antena[indice_receptor]

    # Calcular o ângulo entre os dois nós em relação à direção da antena
    angulo_transmissao = angulo_entre_nos(pos1, direcao_antena1, pos2)
    angulo_rececao = angulo_entre_nos(pos2, direcao_antena2, pos1)

    # Obter os ganhos para transmissão e recepção
    ganho_transmissao = busca_ganhos(angulo_transmissao)
    ganho_rececao = busca_ganhos(angulo_rececao)
    
    return ganho_transmissao, ganho_rececao

# ganho_transmissao, ganho_rececao = calcula_ganhos(posicoes, direcoes_antena, 0, 1, ganhos_df)



def calcula_potencia_recebida(Ptx_dBm, ganho_transmissao, ganho_rececao, d,f):
    """
    Calcula a potência do sinal recebido usando a equação de Friis.
    """
    # L = (4 * np.pi * d / lambda_m)**2
    # Prx_dBm = Ptx_dBm + ganho_transmissao + ganho_rececao - 10 * np.log10(L)
    # return Prx_dBm
    # Calcular a perda de percurso
    if d > 0:
        L = L0 + 10 * gamma * np.log10(d / d0) + np.random.normal(0, sigma)
    else:
        L = L0  # Se a distância for zero, assume-se a perda na distância de referência

    # Calcular a potência do sinal recebido
    Prx_dBm = Ptx_dBm + ganho_transmissao + ganho_rececao - L

    return Prx_dBm

def calcula_potencia_jammer(pos_drone, direcao_antena_drone, pos_jammer, potencia_jammer_dBm, ganhos_df):
    # Calcular a distância entre o drone e o jammer
    d = distancia(pos_drone, pos_jammer)
    
    # Calcular o ângulo entre o drone e o jammer
    angulo_entre_drone_e_jammer = angulo_entre_nos(pos_drone, direcao_antena_drone, pos_jammer)
    
    # Obter o ganho do drone em direção ao jammer
    ganho_drone_para_jammer = busca_ganhos( angulo_entre_drone_e_jammer)
    
    # Calcular a potência do ruído recebido usando a equação de Friis
    L = (4 * np.pi * d / lambda_m) ** 2  # Perda de propagação
    potencia_jammer_recebida_dBm = potencia_jammer_dBm + ganho_drone_para_jammer - 10 * np.log10(L)
    
    return potencia_jammer_recebida_dBm

def calcula_capacidade(potencia_sinal_dBm, potencia_ruido_dBm):
    """
    Calcula a capacidade do canal em bits por segundo usando a fórmula de Shannon-Hartley.
    """
    # Converter potência em dBm para Watts
    potencia_sinal_W = 10 ** (potencia_sinal_dBm / 10) / 1000  # Potência do sinal em Watts
    potencia_ruido_W = 10 ** (potencia_ruido_dBm / 10) / 1000  # Potência do ruído em Watts
    
    # Calcular a razão sinal-ruído (SNR)
    snr = potencia_sinal_W / potencia_ruido_W
    
    # Calcular a capacidade do canal em bits por segundo (bps)
    capacidade = B_Hz * np.log2(1 + snr)
    
    return capacidade/1e3

# -------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------------
for i in range(posicoes.shape[0]):
    for j in range(posicoes.shape[0]):
        if i != j:
            # Calcular a distância entre os dois drones
            d = distancia(posicoes[i], posicoes[j])
            
            # Calcular os ganhos de transmissão e recepção
            ganho_transmissao, ganho_rececao = calcula_ganhos(posicoes, direcoes_antena, i, j, ganhos_df)
            # print(posicoes)
            print('Ganhos Rx,Tx:',ganho_transmissao, ganho_rececao)
            # Calcular a potência do sinal recebido usando a equação de Friis
            potencia_recebida = calcula_potencia_recebida(Ptx_dBm, ganho_transmissao, ganho_rececao, d, lambda_m)
            potencia_ruido = calcula_potencia_jammer(posicoes[j], direcoes_antena[j], posicao_jammer, potencia_jammer_dBm, ganhos_df)
            capacidade_canal=calcula_capacidade(potencia_recebida,potencia_ruido)
            # Imprimir a combinação de nós e a potência do sinal recebido
            print(f"  Potencia recebida do drone {i} para o drone {j}: {potencia_recebida:.2f} dBm")
            print(f"  Ruido recebido no drone {j} e do jammer: {potencia_ruido:.2f} dBm")
            print(f"  Capacidade do canal {i} para {j}: {capacidade_canal:.2f} bps")



COM O DJIKSTRA DENTRO DO AMBIENTE

In [9]:
seed=0

class UAVCommunicationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, num_uavs=num_drones, area_size=(100, 90)):
        super().__init__()
        self.num_uavs = num_uavs
        self.area_size = area_size
        self.posicoes = posicoes.copy()
        self.posicao_jammer = posicao_jammer
        self.ultimo_info = {}

        # Define o espaço de ação: direções das antenas + deslocamentos x e y
        self.action_space = spaces.Box(low=np.array([0]*num_uavs + [-40]*num_uavs + [-40]*num_uavs), 
                                       high=np.array([360]*num_uavs + [40]*num_uavs + [40]*num_uavs), dtype=np.float32)
        self.observation_space = spaces.Box(low=0, high=360, shape=(num_uavs,), dtype=np.float32)

        self.direcoes_antena = np.random.uniform(0, 360, size=num_uavs)
        self.capacidades = []  # Inicializa a lista de capacidades para normalização
        self.subgrade_size=subgrade_size
        self.subgrade_start=subgrade_start
        self.ganhos_df=ganhos_df

    def step(self, action):
        direcoes_antena = action[:self.num_uavs]
        movimentos_x = action[self.num_uavs:2*self.num_uavs]
        movimentos_y = action[2*self.num_uavs:]

        self.direcoes_antena = np.array(direcoes_antena)
        
        for i in range(self.num_uavs):
            nova_pos_x = self.posicoes[i][0] + movimentos_x[i]
            nova_pos_y = self.posicoes[i][1] + movimentos_y[i]
            nova_pos_x = min(max(nova_pos_x, subgrade_start[0]), subgrade_start[0] + subgrade_size[0])
            nova_pos_y = min(max(nova_pos_y, subgrade_start[1]), subgrade_start[1] + subgrade_size[1])
            self.posicoes[i] = np.array([nova_pos_x, nova_pos_y])

        capacidades_por_link = []

        for i in range(self.posicoes.shape[0]):
            for j in range(self.posicoes.shape[0]):
                if i != j:
                    d = distancia(self.posicoes[i], self.posicoes[j])
                    ganho_transmissao, ganho_rececao = calcula_ganhos(self.posicoes, self.direcoes_antena, i, j, ganhos_df)
                    potencia_recebida = calcula_potencia_recebida(Ptx_dBm, ganho_transmissao, ganho_rececao, d, lambda_m)
                    potencia_ruido = calcula_potencia_jammer(self.posicoes[j], self.direcoes_antena[j], self.posicao_jammer, potencia_jammer_dBm, ganhos_df)
                    capacidade_canal = calcula_capacidade(potencia_recebida, potencia_ruido)
                    capacidades_por_link.append(capacidade_canal)

        capacidade_media = np.mean(capacidades_por_link)
        capacidade_minima = np.min(capacidades_por_link)

        self.capacidades.extend(capacidades_por_link)
        if len(self.capacidades) > 1000:
            self.capacidades = self.capacidades[-1000:]

        min_capacidade = np.min(self.capacidades)
        max_capacidade = np.max(self.capacidades)
        # capacidade_media_normalizada = (capacidade_media - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        # capacidade_minima_normalizada = (capacidade_minima - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)

        G = nx.Graph()
        num_drones = self.posicoes.shape[0]
        capacidade_matriz = np.zeros((self.num_uavs, self.num_uavs))
        link_index = 0

        for i in range(self.num_uavs):
            for j in range(self.num_uavs):
                if i != j:
                    capacidade_matriz[i, j] = capacidades_por_link[link_index]
                    link_index += 1

        for i in range(num_drones):
            for j in range(i + 1, num_drones):
                capacidade = capacidade_matriz[i, j]
                G.add_edge(i, j, weight=1.0 / capacidade)

        bottlenecks = {}
        for i in range(num_drones):
            for j in range(num_drones):
                if i != j:
                    try:
                        caminho = nx.dijkstra_path(G, source=i, target=j, weight='weight')
                        menor_capacidade = float('inf')
                        for k in range(len(caminho) - 1):
                            cap_atual = capacidade_matriz[caminho[k], caminho[k+1]]
                            if cap_atual < menor_capacidade:
                                menor_capacidade = cap_atual
                        bottlenecks[(i, j)] = (caminho, menor_capacidade)
                    except nx.NetworkXNoPath:
                        print(f"Não há caminho do nó {i} para o nó {j}")

        capacidade_bottleneck_media = np.mean([bottleneck for caminho, bottleneck in bottlenecks.values()])
        capacidade_bottleneck_minima = np.min([bottleneck for caminho, bottleneck in bottlenecks.values()])

        # capacidade_bottleneck_media_normalizada = (capacidade_bottleneck_media - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        # capacidade_bottleneck_minima_normalizada = (capacidade_bottleneck_minima - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)

        distancia_jammer = np.mean([distancia(pos, self.posicao_jammer) for pos in self.posicoes])
        penalidade_distancia_jammer = -(distancia_jammer ** 2)  # Ajustado para escala apropriada

        distancias_entre_drones = [distancia(self.posicoes[i], self.posicoes[j]) for i in range(num_drones) for j in range(i+1, num_drones)]
        distancia_minima_entre_drones = np.min(distancias_entre_drones)
        penalidade_proximidade_drones = (max(0, (30 - distancia_minima_entre_drones) ** 2))  # Ajustado para escala apropriada

        # Ajuste da função de recompensa para refletir as penalidades não normalizadas
        recompensa = capacidade_media * capacidade_bottleneck_minima - ( 0 * penalidade_distancia_jammer + 0 * penalidade_proximidade_drones)

        capacidades_por_link = [round(capacidade, 3) for capacidade in capacidades_por_link]
        capacidade_media = round(capacidade_media, 3)

        info = {
            'Recompensa': recompensa,
            'Capacidade média [Kbps]': capacidade_media,
            'Capacidades_por_link [Kbps]': capacidades_por_link,
            'Matriz de Capacidades [Kbps]': capacidade_matriz,
            'Capacidade mínima [Kbps]': capacidade_minima,
            # 'Capacidade média normalizada': capacidade_media_normalizada,
            # 'Capacidade mínima normalizada': capacidade_minima_normalizada,
            'Capacidade bottleneck média': capacidade_bottleneck_media,
            'Capacidade bottleneck mínima': capacidade_bottleneck_minima,
            # 'Capacidade bottleneck média normalizada': capacidade_bottleneck_media_normalizada,
            # 'Capacidade bottleneck mínima normalizada': capacidade_bottleneck_minima_normalizada,
            'Penalidade distância ao jammer': penalidade_distancia_jammer,
            'Penalidade proximidade entre drones': penalidade_proximidade_drones
        }

        done = False
        return np.array(self.direcoes_antena), recompensa, done, info

    def render(self, mode='human'):
        sns.set(style="whitegrid")
        fig, ax = plt.subplots(figsize=(8, 6))

        subgrade_start = self.subgrade_start
        subgrade_size = self.subgrade_size
        resolution = 5

        rect = plt.Rectangle(subgrade_start, subgrade_size[0], subgrade_size[1], linewidth=1, edgecolor='blue', facecolor='none')
        ax.add_patch(rect)

        # Desenhar linhas verticais da subgrade dentro do limite da área
        for x in np.arange(subgrade_start[0], subgrade_start[0] + subgrade_size[0] + resolution, resolution):
            ax.axvline(x, color='blue', linestyle='--', linewidth=0.5)

        # Desenhar linhas horizontais da subgrade dentro do limite da área
        for y in np.arange(subgrade_start[1], subgrade_start[1] + subgrade_size[1] + resolution, resolution):
            ax.axhline(y, color='blue', linestyle='--', linewidth=0.5)

        # Desenhar os drones
        for i, pos in enumerate(self.posicoes):
            ax.scatter(pos[0], pos[1], color='blue', s=100, label='Drone' if i == 0 else "")
            ax.text(pos[0], pos[1] - 2, f'{i}', horizontalalignment='center', color='white', fontweight='bold')


        angulos = np.deg2rad(self.ganhos_df['angulo'])
        ganhos = self.ganhos_df['ganho']

        # Normalizar os ganhos para o intervalo [0, 1]
        ganhos_normalizados = (ganhos - ganhos.min()) / (ganhos.max() - ganhos.min())

        # Desenhar o diagrama de radiação sem adicionar ao gráfico principal
        direcao = self.direcoes_antena[0]
        angulos_rotacionados = angulos + np.deg2rad(direcao)
        escala = 6  # Ajuste este valor conforme necessário
        contorno_x = ganhos_normalizados * np.cos(angulos_rotacionados) * escala
        contorno_y = ganhos_normalizados * np.sin(angulos_rotacionados) * escala

        for i, (x, y) in enumerate(self.posicoes):
            direcao = self.direcoes_antena[i]
            angulos_rotacionados = angulos + np.deg2rad(direcao)
            contorno_x = x + ganhos_normalizados * np.cos(angulos_rotacionados) * escala
            contorno_y = y + ganhos_normalizados * np.sin(angulos_rotacionados) * escala
            ax.plot(contorno_x, contorno_y, color='blue', alpha=0.7)

        ax.scatter(self.posicao_jammer[0], self.posicao_jammer[1], color='red', s=100, marker='o', label='Jammer')

        class CustomHandlerPatch(HandlerPatch):
            def create_artists(self, legend, orig_handle, xdescent, ydescent, width, height, fontsize, trans):
                offset_x = width * 0.3  # Ajuste este valor para mover o patch para a direita
                offset_y = ydescent - height * 0.5
                patch = patches.Polygon(
                    np.column_stack((ganhos_normalizados * np.cos(angulos) * escala * 2.5 + offset_x, ganhos_normalizados * np.sin(angulos) * escala * 1.8)), 
                    closed=True, color='blue', alpha=0.9, transform=trans
                )
                return [patch]

        custom_patch = patches.Patch(color='blue', label='Diagrama de Radiação')

        # Adicionar a legenda
        handles = [
            mlines.Line2D([], [], color='blue', marker='o', linestyle='None', markersize=10, label='Drone'),
            mlines.Line2D([], [], color='red', marker='o', linestyle='None', markersize=10, label='Jammer'),
            custom_patch
        ]
        
        for i, pos in enumerate(self.posicoes):
            handles.append(
                mlines.Line2D([], [], color='blue', marker='', linestyle='None', markersize=10,
                            label=f'Drone {i}: Pos ({pos[0]}, {pos[1]}), Dir {self.direcoes_antena[i]:.1f}°')
            )
        
        handles.append(
            mlines.Line2D([], [], color='none', marker='', linestyle='None', markersize=0,
                        label=f'Capacidade Média [bps]: {cap_media:.2f}')
        )
        handles.append(
            mlines.Line2D([], [], color='none', marker='', linestyle='None', markersize=0,
                        label=f'Bottleneck Mínimo [bps]: {bottleneck:.2f}')
        )
        
        # Ajuste do tamanho da legenda com fontsize e disposição horizontal
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), fontsize='small', ncol=3, handles=handles, handler_map={custom_patch: CustomHandlerPatch()})
        # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize='small',borderpad=1, handles=handles, handler_map={custom_patch: CustomHandlerPatch(), (None, None): HandlerTuple()})
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_xlim(0, self.area_size[0])
        ax.set_ylim(0, self.area_size[1])
        ax.set_yticks(np.arange(0, self.area_size[1] + 1, 20))
    
        plt.title(title)

        plt.show()

    def reset(self,seed=None):
        if seed is not None:
            np.random.seed(seed)
        self.direcoes_antena = np.random.uniform(0, 360, size=self.num_uavs)
        self.posicoes = posicoes.copy()
        obs = np.array(self.direcoes_antena)
        return obs, {}

ALGORITMO GENETICO

1.Tamanho da População:

    Maior: Uma população maior oferece uma diversidade genética mais ampla, o que pode ajudar a evitar mínimos locais, mas também aumenta o custo computacional.
    Menor: Uma população menor pode levar a uma convergência mais rápida, mas pode ficar presa em mínimos locais.

2.Taxa de Mutação:

    Alta: Uma taxa de mutação mais alta aumenta a diversidade genética dentro da população e pode ajudar a sair de mínimos locais. No entanto, se for muito alta, pode causar uma busca aleatória, reduzindo a eficiência do algoritmo.
    Baixa: Uma taxa de mutação baixa mantém a estabilidade da população, mas pode resultar em convergência prematura para soluções subótimas.

3.Taxa de Cruzamento:

    Alta: Promove a combinação de características de diferentes indivíduos, potencialmente levando a uma melhoria mais rápida.
    Baixa: Reduz a probabilidade de alterar boas soluções, mas pode limitar a capacidade do algoritmo de explorar novas e potencialmente melhores áreas do espaço de busca.

4.Número de Gerações:

    Maior número de gerações: Permite mais iterações do processo de seleção, cruzamento e mutação, dando mais tempo para a população evoluir.
    Menor número de gerações: Pode ser suficiente se a convergência for rápida, mas corre o risco de não permitir desenvolvimento suficiente, especialmente com uma taxa de mutação baixa.

ALGORITMO GENETICO 

In [10]:
class GeneticAlgorithm:
    def __init__(self, env, grid_points, population_size=500, generations=100, mutation_rate=0.2, crossover_rate=0.9):
        self.env = env
        self.grid_points = grid_points
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate

        self.population = [self.random_chromosome() for _ in range(population_size)]

    def random_chromosome(self):
        indices = np.random.choice(len(self.grid_points), size=self.env.num_uavs, replace=False)
        positions = self.grid_points[indices].flatten()
        directions = np.random.uniform(0, 360, size=self.env.num_uavs)
        return np.concatenate([positions, directions])

    def evaluate_fitness(self, chromosome):
        positions = chromosome[:2*self.env.num_uavs].reshape(self.env.num_uavs, 2)
        directions = chromosome[2*self.env.num_uavs:]
        self.env.posicoes = positions
        self.env.direcoes_antena = directions
        obs, reward, done, info = self.env.step(np.concatenate([directions, np.zeros(self.env.num_uavs), np.zeros(self.env.num_uavs)]))
        return reward, info

    def select(self):
        tournament_size = 5
        selected = []
        fitness_scores = [self.evaluate_fitness(chrom) for chrom in self.population]
        for _ in range(self.population_size):
            contenders = random.sample(list(enumerate(fitness_scores)), tournament_size)
            winner_index = max(contenders, key=lambda item: item[1][0])[0]
            selected.append(self.population[winner_index])
        return selected

    def crossover(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            point = random.randint(1, len(parent1) - 1)
            child1 = np.concatenate([parent1[:point], parent2[point:]])
            child2 = np.concatenate([parent2[:point], parent1[point:]])
            return self.ensure_unique_positions(child1), self.ensure_unique_positions(child2)
        else:
            return parent1, parent2

    def mutate(self, chromosome):
        num_genes = len(chromosome)
        positions = chromosome[:2*self.env.num_uavs].reshape(self.env.num_uavs, 2)
        directions = chromosome[2*self.env.num_uavs:]

        for i in range(self.env.num_uavs):
            if random.random() < self.mutation_rate:
                new_pos_index = np.random.choice(len(self.grid_points))
                while any(np.array_equal(self.grid_points[new_pos_index], positions[j]) for j in range(self.env.num_uavs)):
                    new_pos_index = np.random.choice(len(self.grid_points))
                positions[i] = self.grid_points[new_pos_index]

        for i in range(len(directions)):
            if random.random() < self.mutation_rate:
                mutation = random.uniform(-90, 90)
                directions[i] = (directions[i] + mutation) % 360

        return np.concatenate([positions.flatten(), directions])

    def ensure_unique_positions(self, chromosome):
        positions = chromosome[:2*self.env.num_uavs].reshape(self.env.num_uavs, 2)
        unique_positions = []
        for pos in positions:
            while any(np.array_equal(pos, upos) for upos in unique_positions):
                new_pos_index = np.random.choice(len(self.grid_points))
                pos = self.grid_points[new_pos_index]
            unique_positions.append(pos)
        return np.concatenate([np.array(unique_positions).flatten(), chromosome[2*self.env.num_uavs:]])

    def run(self):
        best_fitness_ever = -np.inf  # Armazenar o melhor fitness encontrado
        best_solution_ever = None    # Armazenar a melhor solução encontrada
        best_info_ever = None        # Armazenar a melhor info correspondente ao melhor fitness

        for generation in range(self.generations):
            new_population = []
            parents = self.select()
            
            while len(new_population) < self.population_size:
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2)
                new_population.extend([self.mutate(child1), self.mutate(child2)])
            
            self.population = new_population[:self.population_size]
            fitness_scores = [self.evaluate_fitness(chrom) for chrom in self.population]
            
            # Encontrar o melhor fitness e sua informação na geração atual
            best_fitness_in_gen, best_info_in_gen = max(fitness_scores, key=lambda x: x[0])
            
            # Verificar se este é o melhor fitness de todas as gerações
            if best_fitness_in_gen > best_fitness_ever:
                best_fitness_ever = best_fitness_in_gen
                best_solution_ever = self.population[fitness_scores.index((best_fitness_in_gen, best_info_in_gen))]
                best_info_ever = best_info_in_gen
                
            print(f"Generation {generation}: Best Fitness = {best_fitness_in_gen:.2f}")
        
        # Retornar a melhor solução e informação encontradas em todas as gerações
        return best_solution_ever, best_info_ever

CORRER O ALGORITMO GENETICO

In [11]:
# # Configuração do ambiente e execução do algoritmo genético
# env = UAVCommunicationEnv()

# # Criar uma instância do Algoritmo Genético
# ga = GeneticAlgorithm(env, grid_points)

# # Executar o algoritmo e obter a melhor solução e informações
# best_solution_genetico, best_info_genetico = ga.run()

# # Renderizar a melhor solução encontrada
# title = 'Direções e Posições Estabelecidas - Algoritmo Genético'
# bottleneck = best_info_genetico['Capacidade bottleneck mínima']
# cap_media = best_info_genetico['Capacidade média [Kbps]']

# # Atualizar o ambiente com a melhor solução antes de renderizar
# env.direcoes_antena = best_solution_genetico
# env.render()

# # Imprimir detalhes da melhor solução encontrada
# print("\nRecompensa:", best_info_genetico['Recompensa'])
# print("\nCapacidade Média:", best_info_genetico['Capacidade média [Kbps]']) 
# print("\nCapacidade bottleneck mínima:", best_info_genetico['Capacidade bottleneck mínima'])  
# print("\nCapacidade bottleneck média:", best_info_genetico['Capacidade bottleneck média'])
# print("\nCapacidade por Link:", best_info_genetico['Capacidades_por_link [Kbps]'])
# print("\nMatriz de Capacidades:", '\n'.join(['\t' + ' '.join(f"{item:.2f}" for item in row) for row in best_info_genetico['Matriz de Capacidades [Kbps]']]))
# capacidade_matriz = best_info_genetico['Matriz de Capacidades [Kbps]']


ALG GENETICO ENCAPSULADO

In [ ]:
class GeneticAlgorithm:
    def __init__(self, env, grid_points, population_size=100, generations=200, mutation_rate=0.15, crossover_rate=0.9):
        self.env = env
        self.grid_points = grid_points
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate

    def random_position_chromosome(self):
        indices = np.random.choice(len(self.grid_points), size=self.env.num_uavs, replace=False)
        return self.grid_points[indices].flatten()

    def random_direction_chromosome(self):
        return np.random.uniform(0, 360, size=self.env.num_uavs)

    def evaluate_position_fitness(self, chromosome):
        positions = chromosome.reshape(self.env.num_uavs, 2)
        self.env.posicoes = positions
        directions = np.zeros(self.env.num_uavs)  # Direções fixas ou predefinidas
        obs, reward, done, info = self.env.step(np.concatenate([directions, np.zeros(self.env.num_uavs), np.zeros(self.env.num_uavs)]))
        return reward, info

    def evaluate_direction_fitness(self, directions):
        self.env.direcoes_antena = directions
        obs, reward, done, info = self.env.step(np.concatenate([directions, np.zeros(self.env.num_uavs), np.zeros(self.env.num_uavs)]))
        return reward, info

    def select(self, population, fitness_scores):
        tournament_size = 5
        selected = []
        for _ in range(self.population_size):
            contenders = random.sample(list(enumerate(fitness_scores)), tournament_size)
            winner_index = max(contenders, key=lambda item: item[1][0])[0]
            selected.append(population[winner_index])
        return selected

    def crossover_positions(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            point = random.randint(1, len(parent1) - 1)
            child1 = np.concatenate([parent1[:point], parent2[point:]])
            child2 = np.concatenate([parent2[:point], parent1[point:]])
            return self.ensure_unique_positions(child1), self.ensure_unique_positions(child2)
        else:
            return parent1, parent2

    def crossover_directions(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            point = random.randint(1, len(parent1) - 1)
            child1 = np.concatenate([parent1[:point], parent2[point:]])
            child2 = np.concatenate([parent2[:point], parent1[point:]])
            return child1, child2
        else:
            return parent1, parent2

    def mutate_positions(self, chromosome):
        positions = chromosome.reshape(self.env.num_uavs, 2)
        for i in range(self.env.num_uavs):
            if random.random() < self.mutation_rate:
                new_pos_index = np.random.choice(len(self.grid_points))
                while any(np.array_equal(self.grid_points[new_pos_index], positions[j]) for j in range(self.env.num_uavs)):
                    new_pos_index = np.random.choice(len(self.grid_points))
                positions[i] = self.grid_points[new_pos_index]
        return positions.flatten()

    def mutate_directions(self, directions):
        for i in range(len(directions)):
            if random.random() < self.mutation_rate:
                mutation = random.uniform(-90, 90)
                directions[i] = (directions[i] + mutation) % 360
        return directions

    def ensure_unique_positions(self, chromosome):
        positions = chromosome.reshape(self.env.num_uavs, 2)
        unique_positions = []
        for pos in positions:
            while any(np.array_equal(pos, upos) for upos in unique_positions):
                new_pos_index = np.random.choice(len(self.grid_points))
                pos = self.grid_points[new_pos_index]
            unique_positions.append(pos)
        return np.array(unique_positions).flatten()

    def run_position_optimization(self):
        best_fitness_ever = -np.inf
        best_positions_ever = None
        best_info_ever = None
        fitness_history_positions = []  # Para armazenar a evolução da recompensa

        population = [self.random_position_chromosome() for _ in range(self.population_size)]

        for generation in range(self.generations):
            fitness_scores = [self.evaluate_position_fitness(chrom) for chrom in population]
            best_fitness_in_gen, best_info_in_gen = max(fitness_scores, key=lambda x: x[0])
            fitness_history_positions.append(best_fitness_in_gen)  # Armazena a melhor recompensa de cada geração

            if best_fitness_in_gen > best_fitness_ever:
                best_fitness_ever = best_fitness_in_gen
                best_positions_ever = population[fitness_scores.index((best_fitness_in_gen, best_info_in_gen))]
                best_info_ever = best_info_in_gen

            selected = self.select(population, fitness_scores)
            new_population = []

            while len(new_population) < self.population_size:
                parent1, parent2 = random.sample(selected, 2)
                child1, child2 = self.crossover_positions(parent1, parent2)
                new_population.extend([self.mutate_positions(child1), self.mutate_positions(child2)])

            population = new_population[:self.population_size]
            print(f"Generation {generation}: Best Fitness = {best_fitness_in_gen:.2f}")

        # Plotar evolução da recompensa durante a otimização de posição
        plt.figure(figsize=(10, 6))
        plt.plot(fitness_history_positions, marker='o', linestyle='-', color='b', label='Best Fitness - Positions')
        plt.title('')
        plt.xlabel('Generation')
        plt.ylabel('')
        plt.grid(True)
        plt.legend()
        plt.show()

        return best_positions_ever, best_info_ever, fitness_history_positions

    def run_direction_optimization(self, best_positions):
        best_fitness_ever = -np.inf
        best_directions_ever = None
        best_info_ever = None
        fitness_history_directions = []  # Para armazenar a evolução da recompensa

        population = [self.random_direction_chromosome() for _ in range(self.population_size)]

        for generation in range(self.generations):
            fitness_scores = [self.evaluate_direction_fitness(chrom) for chrom in population]
            best_fitness_in_gen, best_info_in_gen = max(fitness_scores, key=lambda x: x[0])
            fitness_history_directions.append(best_fitness_in_gen)  # Armazena a melhor recompensa de cada geração

            if best_fitness_in_gen > best_fitness_ever:
                best_fitness_ever = best_fitness_in_gen
                best_directions_ever = population[fitness_scores.index((best_fitness_in_gen, best_info_in_gen))]
                best_info_ever = best_info_in_gen

            selected = self.select(population, fitness_scores)
            new_population = []

            while len(new_population) < self.population_size:
                parent1, parent2 = random.sample(selected, 2)
                child1, child2 = self.crossover_directions(parent1, parent2)
                new_population.extend([self.mutate_directions(child1), self.mutate_directions(child2)])

            population = new_population[:self.population_size]
            print(f"Generation {generation}: Best Fitness = {best_fitness_in_gen:.2f}")

        # Plotar evolução da recompensa durante a otimização de direção
        plt.figure(figsize=(10, 6))
        plt.plot(fitness_history_directions, marker='o', linestyle='-', color='g', label='Best Fitness - Directions')
        plt.axhline(y=1587, color='orange', linestyle='-', label='Best Fitness - Omnidirectional')
        plt.title('')
        plt.xlabel('Generation')
        plt.ylabel('')
        plt.grid(True)
        plt.legend()
        plt.show()

        return best_directions_ever, best_info_ever, fitness_history_directions

# Exemplo de uso do algoritmo dividido em duas fases
env = UAVCommunicationEnv()

# Criar uma instância do Algoritmo Genético
ga = GeneticAlgorithm(env, grid_points)

env.posicao_jammer=[65,40]

env.posicoes=np.array([
    [135, 25],  # Posição do Drone 1
    [150, 25],  # Posição do Drone 2
    [130, 25],  # Posição do Drone 3
    [125, 25]   # Posição do Drone 4
])

env.subgrade_start=[80,20]



# Fase 1: Otimização das Posições
print('---------------------------Descobrir Posições------------------------------')
best_positions, _, fitness_history_positions = ga.run_position_optimization()

print(best_positions)


best_positions=np.array([
    [135, 25],  # Posição do Drone 1
    [150, 25],  # Posição do Drone 2
    [130, 25],  # Posição do Drone 3
    [125, 25]   # Posição do Drone 4
])

# Atualizar as melhores posições no ambiente
env.posicoes = best_positions.reshape(env.num_uavs, 2)

# Fase 2: Otimização das Direções
print('---------------------------Descobrir Direções------------------------------')
best_directions, best_info, fitness_history_directions = ga.run_direction_optimization(best_positions)
print(best_directions)

# best_directions=np.array([293,41,167,168])

# Atualizar as melhores direções no ambiente
env.direcoes_antena = best_directions
env.posicoes=best_positions

# Renderizar a melhor solução encontrada
title = 'Direções e Posições Estabelecidas - Algoritmo SAC'
bottleneck = best_info['Capacidade bottleneck mínima']
cap_media = best_info['Capacidade média [Kbps]']
env.render()

# Imprimir detalhes da melhor solução encontrada
print("\nRecompensa:", best_info['Recompensa'])
print("\nCapacidade Média:", best_info['Capacidade média [Kbps]']) 
print("\nCapacidade bottleneck mínima:", best_info['Capacidade bottleneck mínima'])  
print("\nCapacidade bottleneck média:", best_info['Capacidade bottleneck média'])
print("\nCapacidade por Link:", best_info['Capacidades_por_link [Kbps]'])
print("\nMatriz de Capacidades:", '\n'.join(['\t' + ' '.join(f"{item:.2f}" for item in row) for row in best_info['Matriz de Capacidades [Kbps]']]))
capacidade_matriz = best_info['Matriz de Capacidades [Kbps]']


# Plotar gráfico combinado
plt.figure(figsize=(10, 6))
plt.plot(fitness_history_positions, marker='o', linestyle='-', color='b', label='Best Fitness - Positions')
plt.plot(fitness_history_directions, marker='o', linestyle='-', color='g', label='Best Fitness - Directions')
plt.title('')
plt.xlabel('Generations')
plt.ylabel('')
plt.grid(True)
plt.legend()
plt.show()

PPO

In [17]:
seed=0
import plotly.graph_objects as go

def generate_unique_positions(num_uavs, subgrade_start, subgrade_size, resolution):
    x_values = np.arange(subgrade_start[0], subgrade_start[0] + subgrade_size[0], resolution)
    y_values = np.arange(subgrade_start[1], subgrade_start[1] + subgrade_size[1], resolution)
    possible_positions = np.array(np.meshgrid(x_values, y_values)).T.reshape(-1, 2)
    
    # Escolher posições aleatórias únicas
    indices = np.random.choice(len(possible_positions), size=num_drones, replace=False)
    chosen_positions = possible_positions[indices]
    
    # print("Chosen positions:", chosen_positions)  # Adicionado para depuração
    
    return chosen_positions



class UAVCommunicationEnv_PPO(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, num_uavs=num_drones, area_size=(100, 90)):
        super().__init__()
        self.num_uavs = num_uavs
        self.area_size = area_size
        self.posicoes = posicoes
        self.posicao_jammer = posicao_jammer
        self.ultimo_info = {}

        # Define o espaço de ação: direções das antenas + deslocamentos x e y
        self.action_space = spaces.Box(low=np.array([0]*num_uavs + [-40]*num_uavs + [-40]*num_uavs), 
                                       high=np.array([360]*num_uavs + [40]*num_uavs + [40]*num_uavs), dtype=np.float32)
        self.observation_space = spaces.Box(low=0, high=360, shape=(num_uavs,), dtype=np.float32)

        self.direcoes_antena = np.random.uniform(0, 360, size=num_uavs)
        self.capacidades = []  # Inicializa a lista de capacidades para normalização
        self.subgrade_size=subgrade_size
        self.subgrade_start=subgrade_start
        self.ganhos_df=ganhos_df
        self.resolution=resolution

    def step(self, action):
        
        if self.posicoes.size == 0:
            raise ValueError("As posições dos drones não foram inicializadas corretamente.")
        
        direcoes_antena = action[:self.num_uavs]
        movimentos_x = action[self.num_uavs:2*self.num_uavs]
        movimentos_y = action[2*self.num_uavs:]

        self.direcoes_antena = np.array(direcoes_antena)

        # for i in range(self.num_uavs):
        #     # Aplicar deslocamento
        #     nova_pos_x = self.posicoes[i][0] + movimentos_x[i]
        #     nova_pos_y = self.posicoes[i][1] + movimentos_y[i]

        #     # Forçar nova posição ao ponto mais próximo da grelha
        #     nova_pos_x = np.round(nova_pos_x / resolution) * resolution
        #     nova_pos_y = np.round(nova_pos_y / resolution) * resolution

        #     # Garantir que a nova posição está dentro da subgrade
        #     nova_pos_x = min(max(nova_pos_x, subgrade_start[0]), subgrade_start[0] + subgrade_size[0] - resolution)
        #     nova_pos_y = min(max(nova_pos_y, subgrade_start[1]), subgrade_start[1] + subgrade_size[1] - resolution)

        #     self.posicoes[i] = np.array([nova_pos_x, nova_pos_y])

        
        # Verificar e corrigir duplicações de posições
        unique_positions, indices = np.unique(self.posicoes, axis=0, return_index=True)
        if len(unique_positions) < len(self.posicoes):
            for i in range(len(self.posicoes)):
                if i not in indices:
                    # Se houver duplicatas, gerar uma nova posição válida
                    new_pos = generate_unique_positions(1, self.subgrade_start, self.subgrade_size, self.resolution)[0]
                    while any(np.array_equal(new_pos, pos) for pos in self.posicoes):
                        new_pos = generate_unique_positions(1, self.subgrade_start, self.subgrade_size, self.resolution)[0]
                    self.posicoes[i] = new_pos
        
        capacidades_por_link = []

        for i in range(self.posicoes.shape[0]):
            for j in range(self.posicoes.shape[0]):
                if i != j:
                    d = distancia(self.posicoes[i], self.posicoes[j])
                    ganho_transmissao, ganho_rececao = calcula_ganhos(self.posicoes, self.direcoes_antena, i, j, ganhos_df)
                    potencia_recebida = calcula_potencia_recebida(Ptx_dBm, ganho_transmissao, ganho_rececao, d, lambda_m)
                    potencia_ruido = calcula_potencia_jammer(self.posicoes[j], self.direcoes_antena[j], self.posicao_jammer, potencia_jammer_dBm, ganhos_df)
                    capacidade_canal = calcula_capacidade(potencia_recebida, potencia_ruido)
                    capacidades_por_link.append(capacidade_canal)

        capacidade_media = np.mean(capacidades_por_link)
        capacidade_minima = np.min(capacidades_por_link)

        self.capacidades.extend(capacidades_por_link)
        if len(self.capacidades) > 1000:
            self.capacidades = self.capacidades[-1000:]

        min_capacidade = np.min(self.capacidades)
        max_capacidade = np.max(self.capacidades)
        # capacidade_media_normalizada = (capacidade_media - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        # capacidade_minima_normalizada = (capacidade_minima - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)

        G = nx.Graph()
        num_drones = self.posicoes.shape[0]
        capacidade_matriz = np.zeros((self.num_uavs, self.num_uavs))
        link_index = 0

        for i in range(self.num_uavs):
            for j in range(self.num_uavs):
                if i != j:
                    capacidade_matriz[i, j] = capacidades_por_link[link_index]
                    link_index += 1

        for i in range(num_drones):
            for j in range(i + 1, num_drones):
                capacidade = capacidade_matriz[i, j]
                G.add_edge(i, j, weight=1.0 / capacidade)

        bottlenecks = {}
        for i in range(num_drones):
            for j in range(num_drones):
                if i != j:
                    try:
                        caminho = nx.dijkstra_path(G, source=i, target=j, weight='weight')
                        menor_capacidade = float('inf')
                        for k in range(len(caminho) - 1):
                            cap_atual = capacidade_matriz[caminho[k], caminho[k+1]]
                            if cap_atual < menor_capacidade:
                                menor_capacidade = cap_atual
                        bottlenecks[(i, j)] = (caminho, menor_capacidade)
                    except nx.NetworkXNoPath:
                        print(f"Não há caminho do nó {i} para o nó {j}")

        capacidade_bottleneck_media = np.mean([bottleneck for caminho, bottleneck in bottlenecks.values()])
        capacidade_bottleneck_minima = np.min([bottleneck for caminho, bottleneck in bottlenecks.values()])

        # capacidade_bottleneck_media_normalizada = (capacidade_bottleneck_media - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        # capacidade_bottleneck_minima_normalizada = (capacidade_bottleneck_minima - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)

        distancia_jammer = np.mean([distancia(pos, self.posicao_jammer) for pos in self.posicoes])
        penalidade_distancia_jammer = -(distancia_jammer ** 2)  # Ajustado para escala apropriada

        distancias_entre_drones = [distancia(self.posicoes[i], self.posicoes[j]) for i in range(num_drones) for j in range(i+1, num_drones)]
        distancia_minima_entre_drones = np.min(distancias_entre_drones)
        penalidade_proximidade_drones = (max(0, (30 - distancia_minima_entre_drones) ** 2))  # Ajustado para escala apropriada

        # Ajuste da função de recompensa para refletir as penalidades não normalizadas
        recompensa = capacidade_media * capacidade_bottleneck_minima 

        capacidades_por_link = [round(capacidade, 3) for capacidade in capacidades_por_link]
        capacidade_media = round(capacidade_media, 3)

        info = {
            'Recompensa': recompensa,
            'Capacidade média [Kbps]': capacidade_media,
            'Capacidades_por_link [Kbps]': capacidades_por_link,
            'Matriz de Capacidades [Kbps]': capacidade_matriz,
            'Capacidade mínima [Kbps]': capacidade_minima,
            # 'Capacidade média normalizada': capacidade_media_normalizada,
            # 'Capacidade mínima normalizada': capacidade_minima_normalizada,
            'Capacidade bottleneck média': capacidade_bottleneck_media,
            'Capacidade bottleneck mínima': capacidade_bottleneck_minima,
            # 'Capacidade bottleneck média normalizada': capacidade_bottleneck_media_normalizada,
            # 'Capacidade bottleneck mínima normalizada': capacidade_bottleneck_minima_normalizada,
            'Penalidade distância ao jammer': penalidade_distancia_jammer,
            'Penalidade proximidade entre drones': penalidade_proximidade_drones
        }

        done = False
        truncated=False
        return np.array(self.direcoes_antena), recompensa, done,truncated, info

    def render(self, mode='human'):
        sns.set(style="whitegrid")
        fig, ax = plt.subplots(figsize=(10, 7))

        subgrade_start = self.subgrade_start
        subgrade_size = self.subgrade_size
        resolution = 5

        # Desenhar subgrade com cores mais intensas e linhas finas
        rect = plt.Rectangle(subgrade_start, subgrade_size[0], subgrade_size[1], linewidth=1.5, edgecolor='dodgerblue', facecolor='none')
        ax.add_patch(rect)

        for x in np.arange(subgrade_start[0], subgrade_start[0] + subgrade_size[0] + resolution, resolution):
            ax.axvline(x, color='dodgerblue', linestyle='--', linewidth=0.7, alpha=0.8)
        for y in np.arange(subgrade_start[1], subgrade_start[1] + subgrade_size[1] + resolution, resolution):
            ax.axhline(y, color='dodgerblue', linestyle='--', linewidth=0.7, alpha=0.8)

        # Desenhar os drones com cores distintas
        for i, pos in enumerate(self.posicoes):
            ax.scatter(pos[0], pos[1], color='dimgray', s=200, edgecolor='white', linewidth=1, label='UAV' if i == 0 else "")
            ax.text(
                pos[0], pos[1] + 0.1,  # Ajuste 0.5 para centralizar o texto verticalmente
                f'{i}', 
                horizontalalignment='center', 
                verticalalignment='center', 
                color='white', fontweight='bold'
            )

        # Desenhar setas para indicar a direção de cada drone
        arrow_length = 3  # Comprimento da seta
        for i, (x, y) in enumerate(self.posicoes):
            direcao = np.deg2rad(self.direcoes_antena[i])  # Converte para radianos
            dx = arrow_length * np.cos(direcao)  # Comprimento da seta no eixo X
            dy = arrow_length * np.sin(direcao)  # Comprimento da seta no eixo Y
            ax.quiver(x, y, dx, dy, angles='xy', scale_units='xy', scale=1, color='dimgray', width=0.003)

        # Desenhar o diagrama de radiação usando gradientes
        angulos = np.deg2rad(self.ganhos_df['angulo'])
        ganhos = self.ganhos_df['ganho']
        ganhos_normalizados = (ganhos - ganhos.min()) / (ganhos.max() - ganhos.min())
        escala = 6

        for i, (x, y) in enumerate(self.posicoes):
            direcao = self.direcoes_antena[i]
            angulos_rotacionados = angulos + np.deg2rad(direcao)
            contorno_x = x + ganhos_normalizados * np.cos(angulos_rotacionados) * escala
            contorno_y = y + ganhos_normalizados * np.sin(angulos_rotacionados) * escala
            ax.plot(contorno_x, contorno_y, color='dodgerblue', alpha=0.7, linestyle='-', linewidth=1.2)

        # Desenhar o jammer com um ícone diferente
        ax.scatter(self.posicao_jammer[0], self.posicao_jammer[1], color='red', s=150, marker='X', edgecolor='white', linewidth=1, label='Jammer')

        # Melhorar a legenda com um design organizado e mais legível
        handles = [
            mlines.Line2D([], [], color='dimgray', marker='o', linestyle='None', markersize=10, label='UAV'),
            mlines.Line2D([], [], color='red', marker='X', linestyle='None', markersize=10, label='Jammer')
        ]
        
        for i, pos in enumerate(self.posicoes):
            handles.append(
                mlines.Line2D([], [], color='none', marker='', linestyle='None', markersize=0,
                            label=f'UAV {i}: Pos ({pos[0]}, {pos[1]}), Dir {self.direcoes_antena[i]:.1f}°')
            )

        # Informações adicionais na legenda
        handles.append(
            mlines.Line2D([], [], color='none', marker='', linestyle='None', markersize=0,
                        label=f'Average Capacity [bps]: {cap_media:.2f}')
        )
        handles.append(
            mlines.Line2D([], [], color='none', marker='', linestyle='None', markersize=0,
                        label=f'Bottleneck [bps]: {bottleneck:.2f}')
        )
        
        # Configurar a legenda de forma elegante fora da área de plotagem
        ax.legend(
            loc='upper center', 
            bbox_to_anchor=(0.5, -0.08),  # Ajuste a altura para -0.2
            # fontsize='small', 
            ncol=3,  # Aumenta o número de colunas para que a legenda ocupe mais espaço horizontal
            handles=handles, 
            frameon=True,  # Adicionar borda à legenda para uma aparência mais organizada
            framealpha=0.8,  # Ajusta a transparência da borda da legenda
            fancybox=True,  # Borda arredondada na legenda
            borderpad=1,  # Aumenta o espaçamento interno da legenda
            columnspacing=2.0,  # Aumenta o espaçamento entre as colunas da legenda
            handletextpad=1.5  # Aumenta o espaçamento entre os marcadores e o texto
        )
        
        # Configurar eixos e título
        ax.set_xlabel('X')
        ax.set_ylabel('Y')

        # Definir uma margem de expansão/deslocamento para o eixo X
        expansion_factor_x = 0 # Expande a escala em 10%

        # Calcular os novos limites do eixo X
        x_min = self.subgrade_start[0]
        x_max = self.subgrade_start[0] + self.subgrade_size[0]
        x_range = x_max - x_min
        # ax.set_xlim(x_min - expansion_factor_x * x_range, x_max + expansion_factor_x * x_range)

        # Manter o eixo Y sem alteração
        # ax.set_ylim(y_min, y_max)  # Use os limites que você já definiu para o eixo Y

        # Configurar título do gráfico
        plt.title(title, fontsize=14)

        
        # Ajustar o layout
        plt.tight_layout()

        # Mostrar o gráfico
        plt.show()



    def reset(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
            
        # Gerar posições únicas para os drones
        self.posicoes = generate_unique_positions(self.num_uavs, self.subgrade_start, self.subgrade_size, self.resolution)
        
        
        
        # Inicializar as direções das antenas aleatoriamente
        self.direcoes_antena = np.random.uniform(0, 360, size=self.num_uavs)
        
        # Retornar a observação e informações adicionais
        obs = np.array(self.direcoes_antena)
        info = {}
        return obs, info

PLOTS

PPO COM DATA SET

In [ ]:
import numpy as np
import pandas as pd

# Supondo que seu ambiente seja chamado UAVCommunicationEnv_PPO
env = UAVCommunicationEnv_PPO()

# Número de episódios para gerar o dataset
num_episodes = 1000

# Inicializar uma lista para armazenar os dados
data = []

for episode in range(num_episodes):
    obs, _ = env.reset()
    done = False
    
    # Execute uma etapa para obter a recompensa
    action = env.action_space.sample()  # Ou use seu modelo treinado para escolher a ação
    obs, reward, done, _, info = env.step(action)
    
    # Coleta as posições do jammer, posições dos drones, direções das antenas e a recompensa
    jammer_pos = env.posicao_jammer
    drones_pos = env.posicoes.flatten()  # Flatten para facilitar o armazenamento
    direcoes_antena = env.direcoes_antena  # Direções das antenas
    recompensa = reward

    # Armazenar as informações no formato desejado
    data.append({
        'Jammer_Pos': jammer_pos,
        'Drones_Pos': drones_pos,
        'Direcoes_Antena': direcoes_antena,
        'Recompensa': recompensa
    })

# Converter a lista de dados para um DataFrame do pandas
df = pd.DataFrame(data)

# Expandir as colunas das posições e direções para facilitar a leitura no CSV
df['Jammer_X'], df['Jammer_Y'] = zip(*df['Jammer_Pos'])
df[['Drone_1_X', 'Drone_1_Y', 'Drone_2_X', 'Drone_2_Y', 'Drone_3_X', 'Drone_3_Y', 'Drone_4_X', 'Drone_4_Y']] = pd.DataFrame(df['Drones_Pos'].tolist(), index=df.index)
df[['Direcao_1', 'Direcao_2', 'Direcao_3', 'Direcao_4']] = pd.DataFrame(df['Direcoes_Antena'].tolist(), index=df.index)

# Remover as colunas antigas
df = df.drop(columns=['Jammer_Pos', 'Drones_Pos', 'Direcoes_Antena'])

# Salvar em um arquivo CSV
df.to_csv('DATASET-CENARIO2.csv', index=False)

print(f'Dataset com {num_episodes} episódios salvo em "dataset_treino.csv".')


In [ ]:
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
import torch as th
import matplotlib.pyplot as plt

# Callback personalizado para monitorar recompensas
class RewardCallback(BaseCallback):
    def __init__(self, check_freq: int, verbose: int = 1):
        super(RewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.rewards = []  # Armazena recompensas para plotagem

    def _on_step(self) -> bool:
        # A cada check_freq passos, calcula a recompensa média e salva
        if self.n_calls % self.check_freq == 0:
            # Obter informações do ambiente sobre as recompensas
            ep_rewards = [ep_info['Recompensa'] for ep_info in self.locals['infos'] if 'Recompensa' in ep_info]
            if len(ep_rewards) > 0:
                mean_reward = np.mean(ep_rewards)
                self.rewards.append(mean_reward)
                if self.verbose > 0:
                    print(f"Step {self.num_timesteps}: Mean Reward = {mean_reward:.2f}")
        return True

# Carregar os dados do CSV
df = pd.read_csv('DATASET-CENARIO2.csv')

# Adicionar as direções das antenas no dataset
df['Direcao_1'] = np.random.uniform(0, 360, size=len(df))
df['Direcao_2'] = np.random.uniform(0, 360, size=len(df))
df['Direcao_3'] = np.random.uniform(0, 360, size=len(df))
df['Direcao_4'] = np.random.uniform(0, 360, size=len(df))

# Dados de pré-treinamento: separar as colunas de posições, direções e recompensas
pre_train_data = df[['Jammer_X', 'Jammer_Y',
                     'Drone_1_X', 'Drone_1_Y', 
                     'Drone_2_X', 'Drone_2_Y', 
                     'Drone_3_X', 'Drone_3_Y',
                     'Drone_4_X', 'Drone_4_Y',
                     'Direcao_1', 'Direcao_2', 'Direcao_3', 'Direcao_4',
                     'Recompensa']].values

# Função para criar o ambiente paralelo
def make_env(seed):
    def _init():
        env = UAVCommunicationEnv_PPO()
        env = Monitor(env)
        return env
    return _init

# Definir 1 ambiente paralelo (pode aumentar para n_envs > 1 se desejar)
n_envs = 1
envs = DummyVecEnv([make_env(i) for i in range(n_envs)])

# Função de pré-treinamento (mantida como no código original)
def pre_train(model, envs, pre_train_data, pre_train_iters=100):
    max_pretrain_steps = 20  # Número máximo de passos por episódio durante o pré-treinamento

    for pre_train_iter in range(pre_train_iters):
        print(f"Pre-train iteration {pre_train_iter+1}/{pre_train_iters}")

        for data_index, data in enumerate(pre_train_data):
            print(f"Pre-train data index {data_index+1}/{len(pre_train_data)}")

            # Separar as posições do jammer, posições dos drones, direções das antenas e a recompensa
            jammer_position = data[:2]
            drone_positions = data[2:10].reshape(-1, 2)
            direcoes_antena = data[10:14]
            reward = data[-1]

            # Definir as posições do jammer, dos drones e as direções das antenas no ambiente
            envs.envs[0].unwrapped.posicao_jammer = jammer_position
            envs.envs[0].unwrapped.posicoes = drone_positions
            envs.envs[0].unwrapped.direcoes_antena = direcoes_antena

            # Resetar o ambiente e obter a observação
            obs = envs.reset()[0]

            done = np.zeros(n_envs, dtype=bool)
            step_count = 0  # Contador de passos

            while not np.any(done) & step_count < max_pretrain_steps:
                env.reset()
                # Prever a ação e avaliar
                model.policy.set_training_mode(False)
                predicted_action, _ = model.predict(obs, deterministic=True)
                
                # Converter observação e ação em tensores e garantir que estejam no formato correto
                obs_tensor = th.tensor(obs, dtype=th.float32).to(model.device).unsqueeze(0)  # Adiciona uma dimensão de lote
                predicted_action_tensor = th.tensor(predicted_action, dtype=th.float32).to(model.device).unsqueeze(0)  # Adiciona uma dimensão de lote

                # Realizar o passo no ambiente
                obs, _, done, _ = envs.step([predicted_action] * n_envs)

                # Calcular perdas e atualizar o modelo
                model.policy.set_training_mode(True)
                model.policy.optimizer.zero_grad()
                values, log_prob, entropy = model.policy.evaluate_actions(obs_tensor, predicted_action_tensor)
                policy_loss = -log_prob.mean()
                value_loss = th.nn.functional.mse_loss(values.flatten(), th.tensor([reward] * n_envs, dtype=th.float32).to(model.device))
                loss = policy_loss + value_loss
                loss.backward()
                model.policy.optimizer.step()

                step_count += 1  # Incrementa o contador de passos


# Instanciar o modelo PPO com uma taxa de aprendizado mais segura
modelPPO = PPO("MlpPolicy", envs, verbose=1,
            n_steps=2048,          # Aumenta o número de passos
            batch_size=128,        # Aumenta o tamanho do batch
            learning_rate=1e-4,    # Taxa de aprendizado mais segura
            clip_range=0.2,        # Reduz a faixa de clipping
            ent_coef=0.05)         # Aumenta a entropia para incentivar a exploração

# Criar callback de recompensa
reward_callback = RewardCallback(check_freq=10)

# Pré-treinamento do modelo com base nos dados fornecidos
print(' -------------------------Pré-Treinar o modelo----------------------------')
# pre_train(modelPPO, envs, pre_train_data, pre_train_iters=100)

# Treinamento adicional do modelo com callback para registrar recompensas
modelPPO.learn(total_timesteps=10000, callback=reward_callback)

# Verificar se recompensas foram registradas
if len(reward_callback.rewards) == 0:
    print("Nenhuma recompensa registrada. Verifique o ambiente e o callback.")

# Plotar a evolução da recompensa média ao longo do treinamento
plt.figure(figsize=(10, 6))
plt.plot(np.arange(len(reward_callback.rewards)) * reward_callback.check_freq, reward_callback.rewards, marker='o', linestyle='-', color='b', label='Recompensa Média')
plt.title('Evolução da Recompensa Média durante o Treinamento')
plt.xlabel('Checkpoints')
plt.ylabel('Recompensa Média')
plt.grid(True)
plt.legend()
plt.show()

# Salvar o modelo treinado
modelPPO.save("ppo_uav_communication")


DEEP-QLEARNING


In [25]:
seed=0

def generate_unique_positions(num_uavs, subgrade_start, subgrade_size, resolution):
    x_values = np.arange(subgrade_start[0], subgrade_start[0] + subgrade_size[0], resolution)
    y_values = np.arange(subgrade_start[1], subgrade_start[1] + subgrade_size[1], resolution)
    possible_positions = np.array(np.meshgrid(x_values, y_values)).T.reshape(-1, 2)
    
    # Escolher posições aleatórias únicas
    indices = np.random.choice(len(possible_positions), size=num_drones, replace=False)
    chosen_positions = possible_positions[indices]
    
    # print("Chosen positions:", chosen_positions)  # Adicionado para depuração
    
    return chosen_positions

class UAVCommunicationEnv_DQL(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, num_uavs=num_drones, area_size=(100, 90)):
        super().__init__()
        self.num_uavs = num_uavs
        self.area_size = area_size
        self.posicoes = posicoes.copy()
        self.posicao_jammer = posicao_jammer
        self.ultimo_info = {}

        # Define o espaço de ação: direções das antenas + deslocamentos x e y
        self.action_space = spaces.Box(low=np.array([0]*num_uavs + [-40]*num_uavs + [-40]*num_uavs), 
                                       high=np.array([360]*num_uavs + [40]*num_uavs + [40]*num_uavs), dtype=np.float32)
        self.observation_space = spaces.Box(low=0, high=360, shape=(num_uavs,), dtype=np.float32)

        self.direcoes_antena = np.random.uniform(0, 360, size=num_uavs)
        self.capacidades = []  # Inicializa a lista de capacidades para normalização
        self.subgrade_size=subgrade_size
        self.subgrade_start=subgrade_start
        self.ganhos_df=ganhos_df
        self.resolution=resolution

    def step(self, action):
        direcoes_antena = action[:self.num_uavs]
        movimentos_x = action[self.num_uavs:2*self.num_uavs]
        movimentos_y = action[2*self.num_uavs:]

        self.direcoes_antena = np.random.uniform(0, 360, size=self.num_uavs)
        
        
        # Deslocamento aleatório
        deslocamento_x = np.random.uniform(-10, 10, size=self.num_uavs).astype(np.int32)
        deslocamento_y = np.random.uniform(-10, 10, size=self.num_uavs).astype(np.int32)

        # Atualiza as posições com base nas posições anteriores mais o deslocamento
        self.posicoes[:, 0] += deslocamento_x  # Atualiza a posição x dos drones
        self.posicoes[:, 1] += deslocamento_y  # Atualiza a posição y dos drones
        self.posicoes = generate_unique_positions(self.num_uavs, self.subgrade_start, self.subgrade_size, self.resolution)
        
        
        
        # for i in range(self.num_uavs):
        #     # Aplicar deslocamento
        #     nova_pos_x = self.posicoes[i][0] + movimentos_x[i]
        #     nova_pos_y = self.posicoes[i][1] + movimentos_y[i]

        #     # Forçar nova posição ao ponto mais próximo da grelha
        #     nova_pos_x = np.round(nova_pos_x / resolution) * resolution
        #     nova_pos_y = np.round(nova_pos_y / resolution) * resolution

        #     # Garantir que a nova posição está dentro da subgrade
        #     nova_pos_x = min(max(nova_pos_x, subgrade_start[0]), subgrade_start[0] + subgrade_size[0] - resolution)
        #     nova_pos_y = min(max(nova_pos_y, subgrade_start[1]), subgrade_start[1] + subgrade_size[1] - resolution)

        #     self.posicoes[i] = np.array([nova_pos_x, nova_pos_y])

        
        # Verificar e corrigir duplicações de posições
        unique_positions, indices = np.unique(self.posicoes, axis=0, return_index=True)
        if len(unique_positions) < len(self.posicoes):
            for i in range(len(self.posicoes)):
                if i not in indices:
                    # Se houver duplicatas, gerar uma nova posição válida
                    new_pos = generate_unique_positions(1, self.subgrade_start, self.subgrade_size, self.resolution)[0]
                    while any(np.array_equal(new_pos, pos) for pos in self.posicoes):
                        new_pos = generate_unique_positions(1, self.subgrade_start, self.subgrade_size, self.resolution)[0]
                    self.posicoes[i] = new_pos

        capacidades_por_link = []

        for i in range(self.posicoes.shape[0]):
            for j in range(self.posicoes.shape[0]):
                if i != j:
                    d = distancia(self.posicoes[i], self.posicoes[j])
                    ganho_transmissao, ganho_rececao = calcula_ganhos(self.posicoes, self.direcoes_antena, i, j, ganhos_df)
                    potencia_recebida = calcula_potencia_recebida(Ptx_dBm, ganho_transmissao, ganho_rececao, d, lambda_m)
                    potencia_ruido = calcula_potencia_jammer(self.posicoes[j], self.direcoes_antena[j], self.posicao_jammer, potencia_jammer_dBm, ganhos_df)
                    capacidade_canal = calcula_capacidade(potencia_recebida, potencia_ruido)
                    capacidades_por_link.append(capacidade_canal)

        capacidade_media = np.mean(capacidades_por_link)
        capacidade_minima = np.min(capacidades_por_link)

        self.capacidades.extend(capacidades_por_link)
        if len(self.capacidades) > 1000:
            self.capacidades = self.capacidades[-1000:]

        min_capacidade = np.min(self.capacidades)
        max_capacidade = np.max(self.capacidades)
        # capacidade_media_normalizada = (capacidade_media - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        # capacidade_minima_normalizada = (capacidade_minima - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)

        G = nx.Graph()
        num_drones = self.posicoes.shape[0]
        capacidade_matriz = np.zeros((self.num_uavs, self.num_uavs))
        link_index = 0

        for i in range(self.num_uavs):
            for j in range(self.num_uavs):
                if i != j:
                    capacidade_matriz[i, j] = capacidades_por_link[link_index]
                    link_index += 1

        for i in range(num_drones):
            for j in range(i + 1, num_drones):
                capacidade = capacidade_matriz[i, j]
                G.add_edge(i, j, weight=1.0 / capacidade)

        bottlenecks = {}
        for i in range(num_drones):
            for j in range(num_drones):
                if i != j:
                    try:
                        caminho = nx.dijkstra_path(G, source=i, target=j, weight='weight')
                        menor_capacidade = float('inf')
                        for k in range(len(caminho) - 1):
                            cap_atual = capacidade_matriz[caminho[k], caminho[k+1]]
                            if cap_atual < menor_capacidade:
                                menor_capacidade = cap_atual
                        bottlenecks[(i, j)] = (caminho, menor_capacidade)
                    except nx.NetworkXNoPath:
                        print(f"Não há caminho do nó {i} para o nó {j}")

        capacidade_bottleneck_media = np.mean([bottleneck for caminho, bottleneck in bottlenecks.values()])
        capacidade_bottleneck_minima = np.min([bottleneck for caminho, bottleneck in bottlenecks.values()])

        # capacidade_bottleneck_media_normalizada = (capacidade_bottleneck_media - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        # capacidade_bottleneck_minima_normalizada = (capacidade_bottleneck_minima - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)

        distancia_jammer = np.mean([distancia(pos, self.posicao_jammer) for pos in self.posicoes])
        penalidade_distancia_jammer = -(distancia_jammer ** 2)  # Ajustado para escala apropriada

        distancias_entre_drones = [distancia(self.posicoes[i], self.posicoes[j]) for i in range(num_drones) for j in range(i+1, num_drones)]
        distancia_minima_entre_drones = np.min(distancias_entre_drones)
        penalidade_proximidade_drones = (max(0, (30 - distancia_minima_entre_drones) ** 2))  # Ajustado para escala apropriada

        # Ajuste da função de recompensa para refletir as penalidades não normalizadas
        recompensa = capacidade_media * capacidade_bottleneck_minima - ( 0 * penalidade_distancia_jammer + 0 * penalidade_proximidade_drones)

        capacidades_por_link = [round(capacidade, 3) for capacidade in capacidades_por_link]
        capacidade_media = round(capacidade_media, 3)

        info = {
            'Recompensa': recompensa,
            'Capacidade média [Kbps]': capacidade_media,
            'Capacidades_por_link [Kbps]': capacidades_por_link,
            'Matriz de Capacidades [Kbps]': capacidade_matriz,
            'Capacidade mínima [Kbps]': capacidade_minima,
            # 'Capacidade média normalizada': capacidade_media_normalizada,
            # 'Capacidade mínima normalizada': capacidade_minima_normalizada,
            'Capacidade bottleneck média': capacidade_bottleneck_media,
            'Capacidade bottleneck mínima': capacidade_bottleneck_minima,
            # 'Capacidade bottleneck média normalizada': capacidade_bottleneck_media_normalizada,
            # 'Capacidade bottleneck mínima normalizada': capacidade_bottleneck_minima_normalizada,
            'Penalidade distância ao jammer': penalidade_distancia_jammer,
            'Penalidade proximidade entre drones': penalidade_proximidade_drones
        }

        done = False
        return np.array(self.direcoes_antena), recompensa, done, info

    def render(self, mode='human'):
        sns.set(style="whitegrid")
        fig, ax = plt.subplots(figsize=(8, 6))

        subgrade_start = self.subgrade_start
        subgrade_size = self.subgrade_size
        resolution = 5

        rect = plt.Rectangle(subgrade_start, subgrade_size[0], subgrade_size[1], linewidth=1, edgecolor='blue', facecolor='none')
        ax.add_patch(rect)

        # Desenhar linhas verticais da subgrade dentro do limite da área
        for x in np.arange(subgrade_start[0], subgrade_start[0] + subgrade_size[0] + resolution, resolution):
            ax.axvline(x, color='blue', linestyle='--', linewidth=0.5)

        # Desenhar linhas horizontais da subgrade dentro do limite da área
        for y in np.arange(subgrade_start[1], subgrade_start[1] + subgrade_size[1] + resolution, resolution):
            ax.axhline(y, color='blue', linestyle='--', linewidth=0.5)

        # Desenhar os drones
        for i, pos in enumerate(self.posicoes):
            ax.scatter(pos[0], pos[1], color='blue', s=100, label='Drone' if i == 0 else "")
            ax.text(pos[0], pos[1] - 2, f'{i}', horizontalalignment='center', color='white', fontweight='bold')


        angulos = np.deg2rad(self.ganhos_df['angulo'])
        ganhos = self.ganhos_df['ganho']

        # Normalizar os ganhos para o intervalo [0, 1]
        ganhos_normalizados = (ganhos - ganhos.min()) / (ganhos.max() - ganhos.min())

        # Desenhar o diagrama de radiação sem adicionar ao gráfico principal
        direcao = self.direcoes_antena[0]
        angulos_rotacionados = angulos + np.deg2rad(direcao)
        escala = 6  # Ajuste este valor conforme necessário
        contorno_x = ganhos_normalizados * np.cos(angulos_rotacionados) * escala
        contorno_y = ganhos_normalizados * np.sin(angulos_rotacionados) * escala

        for i, (x, y) in enumerate(self.posicoes):
            direcao = self.direcoes_antena[i]
            angulos_rotacionados = angulos + np.deg2rad(direcao)
            contorno_x = x + ganhos_normalizados * np.cos(angulos_rotacionados) * escala
            contorno_y = y + ganhos_normalizados * np.sin(angulos_rotacionados) * escala
            ax.plot(contorno_x, contorno_y, color='blue', alpha=0.7)

        ax.scatter(self.posicao_jammer[0], self.posicao_jammer[1], color='red', s=100, marker='o', label='Jammer')

        class CustomHandlerPatch(HandlerPatch):
            def create_artists(self, legend, orig_handle, xdescent, ydescent, width, height, fontsize, trans):
                offset_x = width * 0.3  # Ajuste este valor para mover o patch para a direita
                offset_y = ydescent - height * 0.5
                patch = patches.Polygon(
                    np.column_stack((ganhos_normalizados * np.cos(angulos) * escala * 2.5 + offset_x, ganhos_normalizados * np.sin(angulos) * escala * 1.8)), 
                    closed=True, color='blue', alpha=0.9, transform=trans
                )
                return [patch]

        custom_patch = patches.Patch(color='blue', label='Diagrama de Radiação')

        # Adicionar a legenda
        handles = [
            mlines.Line2D([], [], color='blue', marker='o', linestyle='None', markersize=10, label='Drone'),
            mlines.Line2D([], [], color='red', marker='o', linestyle='None', markersize=10, label='Jammer'),
            custom_patch
        ]
        
        for i, pos in enumerate(self.posicoes):
            handles.append(
                mlines.Line2D([], [], color='blue', marker='', linestyle='None', markersize=10,
                            label=f'Drone {i}: Pos ({pos[0]}, {pos[1]}), Dir {self.direcoes_antena[i]:.1f}°')
            )
        
        handles.append(
            mlines.Line2D([], [], color='none', marker='', linestyle='None', markersize=0,
                        label=f'Capacidade Média [bps]: {cap_media:.2f}')
        )
        handles.append(
            mlines.Line2D([], [], color='none', marker='', linestyle='None', markersize=0,
                        label=f'Bottleneck Mínimo [bps]: {bottleneck:.2f}')
        )
        
        # Ajuste do tamanho da legenda com fontsize e disposição horizontal
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), fontsize='small', ncol=3, handles=handles, handler_map={custom_patch: CustomHandlerPatch()})
        # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize='small',borderpad=1, handles=handles, handler_map={custom_patch: CustomHandlerPatch(), (None, None): HandlerTuple()})
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_xlim(0, self.area_size[0])
        ax.set_ylim(0, self.area_size[1])
        ax.set_yticks(np.arange(0, self.area_size[1] + 1, 20))
    
        plt.title(title)

        plt.show()

    def reset(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
            
        # Gerar posições únicas para os drones
        self.posicoes=posicoes
        # self.posicoes = generate_unique_positions(self.num_uavs, self.subgrade_start, self.subgrade_size, self.resolution)
        
        
        
        # Inicializar as direções das antenas aleatoriamente
        self.direcoes_antena = direcoes_antena
        
        # Retornar a observação e informações adicionais
        obs = np.array(self.direcoes_antena)
        info = {}
        return obs, info

SOFT ACTOR CRITIC

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque
import os
import matplotlib.pyplot as plt

# Rede neural para os críticos (Q-networks)
class Critic(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 512)
        self.fc5 = nn.Linear(512, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        return self.fc5(x)

# Rede neural para o ator (policy network)
class Actor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 512)
        self.mu_layer = nn.Linear(512, output_dim)
        self.log_std_layer = nn.Linear(512, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        mu = self.mu_layer(x)
        log_std = self.log_std_layer(x)
        log_std = torch.clamp(log_std, -20, 2)  # Para estabilidade numérica
        return mu, log_std

    def sample(self, state):
        mu, log_std = self.forward(state)
        std = log_std.exp()
        normal = torch.distributions.Normal(mu, std)
        z = normal.rsample()  # Reparametrização
        action = torch.tanh(z)
        log_prob = normal.log_prob(z) - torch.log(1 - action.pow(2) + 1e-6)
        return action, log_prob.sum(dim=-1)


class ReplayBuffer:
    def __init__(self, max_size=100000):
        self.buffer = deque(maxlen=max_size)

    def add(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def size(self):
        return len(self.buffer)


class SACAgent:
    def __init__(self, state_dim, action_dim, action_range):
        self.actor = Actor(state_dim, action_dim)
        self.critic1 = Critic(state_dim + action_dim, 1)
        self.critic2 = Critic(state_dim + action_dim, 1)
        self.target_critic1 = Critic(state_dim + action_dim, 1)
        self.target_critic2 = Critic(state_dim + action_dim, 1)
        
        # Verifique as dimensões dos estados e ações
        print(f"State dim: {state_dim}, Action dim: {action_dim}")
        
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=1e-4)
        self.critic1_optimizer = optim.Adam(self.critic1.parameters(), lr=1e-4)
        self.critic2_optimizer = optim.Adam(self.critic2.parameters(), lr=1e-4)

        self.target_critic1.load_state_dict(self.critic1.state_dict())
        self.target_critic2.load_state_dict(self.critic2.state_dict())

        self.replay_buffer = ReplayBuffer()
        self.gamma = 0.99
        self.tau = 0.01
        self.alpha = 0.001  # Entropy coefficient
        self.action_range = action_range
        
        self.reward_history = []
        self.min_reward = float('inf')
        self.max_reward = -float('inf')

    def update_reward_statistics(self, reward):
        self.reward_history.append(reward)
        if reward < self.min_reward:
            self.min_reward = reward
        if reward > self.max_reward:
            self.max_reward = reward

    def normalize_reward(self, reward):
        self.update_reward_statistics(reward)
        if self.max_reward == self.min_reward:
            return 0.0  # Evita divisão por zero
        return (reward - self.min_reward) / (self.max_reward - self.min_reward)

    def save(self, filepath):
        # Salvar os modelos dos atores e críticos
        torch.save({
            'actor_state_dict': self.actor.state_dict(),
            'critic1_state_dict': self.critic1.state_dict(),
            'critic2_state_dict': self.critic2.state_dict(),
            'actor_optimizer_state_dict': self.actor_optimizer.state_dict(),
            'critic1_optimizer_state_dict': self.critic1_optimizer.state_dict(),
            'critic2_optimizer_state_dict': self.critic2_optimizer.state_dict(),
        }, filepath)
        print(f"Model saved to {filepath}")

    def load(self, filepath):
        # Carregar os modelos dos atores e críticos
        checkpoint = torch.load(filepath)
        self.actor.load_state_dict(checkpoint['actor_state_dict'])
        self.critic1.load_state_dict(checkpoint['critic1_state_dict'])
        self.critic2.load_state_dict(checkpoint['critic2_state_dict'])
        self.actor_optimizer.load_state_dict(checkpoint['actor_optimizer_state_dict'])
        self.critic1_optimizer.load_state_dict(checkpoint['critic1_optimizer_state_dict'])
        self.critic2_optimizer.load_state_dict(checkpoint['critic2_optimizer_state_dict'])
        print(f"Model loaded from {filepath}")
    
    
    def select_action(self, state):
        state = torch.FloatTensor(state).unsqueeze(0)  # Certifique-se de que o estado é um vetor e converta para tensor
        action, _ = self.actor.sample(state)
        return action.detach().numpy().flatten()

    def update(self, batch_size):
        if self.replay_buffer.size() < batch_size:
            return

        samples = self.replay_buffer.sample(batch_size)
        states, actions, rewards, next_states, dones = zip(*samples)
        states = torch.FloatTensor(np.array(states))
        actions = torch.FloatTensor(np.array(actions))
        rewards = torch.FloatTensor(rewards).unsqueeze(1)
        next_states = torch.FloatTensor(np.array(next_states))
        dones = torch.FloatTensor(dones).unsqueeze(1)

        with torch.no_grad():
            next_actions, next_log_probs = self.actor.sample(next_states)
            target_q1 = self.target_critic1(torch.cat([next_states, next_actions], 1))
            target_q2 = self.target_critic2(torch.cat([next_states, next_actions], 1))
            target_q = rewards + (1 - dones) * self.gamma * (torch.min(target_q1, target_q2) - self.alpha * next_log_probs.unsqueeze(1))

        q1 = self.critic1(torch.cat([states, actions], 1))
        q2 = self.critic2(torch.cat([states, actions], 1))
        critic1_loss = nn.MSELoss()(q1, target_q)
        critic2_loss = nn.MSELoss()(q2, target_q)

        self.critic1_optimizer.zero_grad()
        critic1_loss.backward()
        self.critic1_optimizer.step()

        self.critic2_optimizer.zero_grad()
        critic2_loss.backward()
        self.critic2_optimizer.step()

        actions, log_probs = self.actor.sample(states)
        q1_pi = self.critic1(torch.cat([states, actions], 1))
        q2_pi = self.critic2(torch.cat([states, actions], 1))
        actor_loss = (self.alpha * log_probs - torch.min(q1_pi, q2_pi)).mean()

        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Update target networks
        for target_param, param in zip(self.target_critic1.parameters(), self.critic1.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

        for target_param, param in zip(self.target_critic2.parameters(), self.critic2.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)


# Defina o ambiente e os parâmetros de treinamento
env = UAVCommunicationEnv_DQL()  # Substitua pelo seu ambiente
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_range = [env.action_space.low, env.action_space.high]
num_episodes = 2000
batch_size = 256

# Inicialize o agente
agent = SACAgent(state_dim, action_dim, action_range)

# Variáveis para monitorar o desempenho
reward_history = []
average_rewards = []

# Loop de treinamento
for episode in range(num_episodes):
    state,_ = env.reset()
    print(f"Estado inicial após reset: {state}, Dimensão: {len(state)}")  # Debugging
    episode_reward = 0
    done = False

    while not done:
        # Seleciona ação e executa no ambiente
        action = agent.select_action(state)
        if len(state) != state_dim:
            raise ValueError(f"Dimensão do estado ({len(state)}) não corresponde a state_dim ({state_dim}). Verifique o ambiente.")
        next_state, reward, done, info = env.step(action)

        # Armazena experiência no buffer
        agent.replay_buffer.add((state, action, reward, next_state, done))

        # Atualiza o modelo a cada interação, se o buffer tiver o tamanho suficiente
        if agent.replay_buffer.size() > batch_size:
            agent.update(batch_size)

        # Atualiza o estado e o episódio de recompensa acumulada
        state = next_state
        episode_reward += reward
        done =True
        
        

    # Armazena recompensas e calcula a média a cada 50 episódios
    reward_history.append(episode_reward)
    if (episode + 1) % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        average_rewards.append(avg_reward)
        print(f"Episódio {episode + 1}: Recompensa média dos últimos 50 episódios: {avg_reward}")

# Salve o modelo treinado
agent.save("sac_uav_model.pth")







In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(50, num_episodes + 1, 50), average_rewards_normalized, linestyle='-', color='b', label='Average Reward')
# plt.plot(mean_timesteps_array, mean_rewards, linestyle='-', color='b', label='Average Reward')
plt.xlabel('Episodes')
plt.ylabel('')
plt.title('')
plt.show()

In [ ]:
agent = SACAgent(state_dim, action_dim, action_range)
agent.load("C:/Users/tass/OneDrive - EXÉRCITO PORTUGUÊS/Ambiente de Trabalho/Tese/Simulaçãoes/best_models/sac_best_model.pth")
env=UAVCommunicationEnv_DQL()
env.reset()
# Inicializar variáveis para armazenar a melhor recompensa e o melhor episódio
best_reward = -float('inf')
best_episode = None
best_episode_steps = []

# Executar o modelo no ambiente 100 vezes
num_test_episodes = 1000

for episode in range(num_test_episodes):
    state, _ = env.reset()
    episode_reward = 0
    episode_steps = []

    
    action = agent.select_action(state)  # Seleciona ação sem exploração adicional
    next_state, reward, done, info = env.step(action)
    episode_steps.append((state, action))  # Armazena o estado e a ação
    state = next_state
    episode_reward += reward

    # print(f"Test Episode: {episode}, Reward: {episode_reward}")

    # Se este episódio for o melhor até agora, salvá-lo
    if episode_reward > best_reward:
        best_reward = episode_reward
        best_episode = episode
        best_episode_steps = episode_steps

# Renderizar o melhor episódio
print(f"Best Episode: {best_episode}, Best Reward: {best_reward}")

# Resetar o ambiente
state, _ = env.reset()
# env.render()

for step in best_episode_steps:
    state, action = step
    env.step(action)
    obs, reward, done, info = env.step(action)
    title = 'Direções e Posições Estabelecidas - Algoritmo SAC'
    bottleneck = info['Capacidade bottleneck mínima']
    cap_media = info['Capacidade média [Kbps]']
    print(info['Capacidades_por_link [Kbps]'])
    env.render()

print("Renderização do melhor episódio concluída.")